In [15]:
import pickle

with open('/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    df = pickle.load(file)

df['protein_id'] = df['pdb_id'] + df['struct_asym_id']
df[df['pdb_id'] == '1cf4']

,ACC,SEG,ID,up_aas,n_aas,SEG_ID,SITE_NAME,pdb_id,auth_asym_id,struct_asym_id,rep_chain,RoG,aas,centre,SASA,VOL,protein_id


seems like ligysis uses label_seq labeling, not auth! Same for P2Rank predictions!

# loop over PDB structures
go over PDB and read input indexes.

In [7]:
import sys
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import cryptoshow_utils

binding_sites = {}
for protein_id, binding_site in zip(df['protein_id'], df['aas']):
    if protein_id not in binding_sites:
        binding_sites[protein_id] = []
    binding_sites[protein_id].append(binding_site)

# Translate from author numbering to mmCIF numbering (zero-based)
translated_binding_sites = {}
for protein_id in binding_sites:
    pdb_id, chain_id = protein_id[:4], protein_id[4:]
    translated_binding_sites[protein_id] = []
    for binding_site in binding_sites[protein_id]:
        translated_binding_sites[protein_id].append(cryptoshow_utils.map_auth_to_mmcif_numbering(pdb_id, chain_id, binding_site, auth=False))

In [ ]:
with open("/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_without_unobserved.csv", "w") as file:
    for protein_id in translated_binding_sites:
        binding_residues = set()
        for binding_site, sequence in translated_binding_sites[protein_id]:
            binding_residues.update(binding_site)
        file.write(f'{protein_id};UNKNOWN;NON_CRYPTIC;{" ".join(sorted(binding_residues))};{sequence}\n')

In [22]:

def get_distance_matrix(pdb_id, chain_id):
    from scipy.spatial import distance_matrix

    coords = get_coordinates(pdb_id, chain_id)
    dist_matrix = distance_matrix(coords, coords)

    return dist_matrix

def get_coordinates(pdb_id, chain_id, auth=True):
    import biotite.database.rcsb as rcsb
    import biotite.structure.io.pdbx as pdbx
    from biotite.structure.io.pdbx import get_structure
    from biotite.structure import get_residues
    
    cif_file_path = rcsb.fetch(pdb_id, "cif", cryptoshow_utils.CIF_FILES_PATH)
    cif_file = pdbx.CIFFile.read(cif_file_path)
    
    protein = get_structure(cif_file, model=1, use_author_fields=auth)
    protein = protein[(protein.atom_name == "CA") 
                        & (protein.element == "C") 
                        & (protein.chain_id == chain_id) ]
    residue_ids, residue_types = get_residues(protein)

    coords = protein.coord

    assert len(residue_ids) == len(coords) == len(residue_types), "Number of residues and coordinates do not match"

    return coords

import numpy as np
COORDINATES_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'
for protein_id in translated_binding_sites:
    pdb_id, chain_id = protein_id[:4], protein_id[4:]
    coordinates = get_coordinates(pdb_id, chain_id, auth=False)
    np.save(f'{COORDINATES_PATH}/{protein_id}.npy', coordinates)

In [23]:
df['pdb_id'].nunique()

3376